# *VBN opto - Nov 2023*
<img src="" width="380" />

In [ ]:
%env AIBS_RIG_ID=NP2

## Run the update/reset shortcut on the desktop before each experiment
***
***
# **Without mouse on stage**

In [ ]:
import contextlib
import time

import np_config
import np_jobs
import np_logging
import np_services
import np_session
import np_workflows
from np_workflows import npxc
import np_workflows.experiments.openscope_barcode as Barcode

from np_services.resources.zro import ZroError 
import contextlib

logger = np_logging.getLogger()

np_workflows.elapsed_time_widget()

***
## Quiet mode
**on**  [*default*]
- error details are hidden
- regular messages displayed (log level = INFO)

**off**
- full error details (with traceback)
- extra messages displayed (log level = DEBUG)

In [ ]:
np_workflows.quiet_mode_widget()

***
## Launch apps via RSC
[optional]

In [ ]:
with contextlib.suppress(Exception):
    np_services.start_rsc_apps()

***
## Select mouse and user

In [ ]:
user, mouse = np_workflows.user_and_mouse_widget()

***
## Check MTrain and select workflow
Re-run cell this cell if mouse ID is changed

In [ ]:
np_workflows.mtrain_widget(mouse)

In [ ]:
import enum
import functools
import json
import pathlib
from typing import Literal

class Workflow(enum.Enum):
    """Enum for the different session types available .
    - can control workflow and paramater sets
    """
    PRETEST = "pretest"
    HAB = "hab"
    EPHYS = "ephys"
      
class VBNMixin:
    """Provides project-specific methods and attributes, mainly related to camstim scripts."""
    
    workflow: Workflow
    """Enum for particular workflow/session, e.g. PRETEST, HAB_60, HAB_90,
    EPHYS."""
    
    session: np_session.PipelineSession
    mouse: np_session.Mouse
    user: np_session.User
    platform_json: np_session.PlatformJson
    camstim_script_path: str = '//allen/programs/mindscope/workgroups/dynamicrouting/docOpto/run_script.py'
    
    @property
    def camstim_params(self) -> dict[str, str]:
        dict(json_path=self.camstim_params_path)
        
    @property
    def camstim_params_path(self) -> str:
        return f'//allen/programs/mindscope/workgroups/dynamicrouting/docOpto/laser_script_params_HVAsilencing_{self.mouse}_{self.ephys_day}.json'
        
    @property
        
    @property 
    def ephys_day(self) -> int:
        return self.mouse.state.get('last_ephys_day', 0) + 1
    
    @property
    def recorders(self) -> tuple[np_services.Service, ...]:
        """Services to be started before stimuli run, and stopped after. Session-dependent."""
        match self.workflow:
            case Workflow.PRETEST | Workflow.EPHYS:
                return (np_services.Sync, np_services.VideoMVR, np_services.OpenEphys)
            case Workflow.HAB:
                return (np_services.Sync, np_services.VideoMVR)

    @property
    def stims(self) -> tuple[np_services.Service, ...]:
        return (np_services.ScriptCamstim, )
    
    def initialize_and_test_services(self) -> None:
        """Configure, initialize (ie. reset), then test all services."""
        
        np_services.MouseDirector.user = self.user.id
        np_services.MouseDirector.mouse = self.mouse.id

        np_services.OpenEphys.folder = self.session.folder

        np_services.NewScaleCoordinateRecorder.log_root = self.session.npexp_path
        np_services.NewScaleCoordinateRecorder.log_name = self.platform_json.path.name

        assert pathlib.Path(self.camstim_script_path).exists(), f"Filepath does not exist or is not accessible: {self.camstim_script_path=}"
        assert pathlib.Path(self.camstim_params_path).exists(), f"Filepath does not exist or is not accessible: {self.camstim_params_path=}"
        
        np_services.ScriptCamstim.script = self.camstim_script_path
        np_services.ScriptCamstim.params = self.camstim_params
        
        self.configure_services()

        super().initialize_and_test_services()

    def update_state(self) -> None:
        "Store useful but non-essential info."
        self.mouse.state['last_session'] = self.session.id
        self.mouse.state['last_vbn_session'] = str(self.workflow)
        self.mouse.state['last_ephys_day'] = self.ephys_day
        if self.mouse == 366122:
            return
        match self.workflow:
            case Workflow.PRETEST:
                return
            case Workflow.HAB:
                self.session.project.state['latest_hab'] = self.session.id
            case Workflow.EPHYS:
                self.session.project.state['latest_ephys'] = self.session.id
                self.session.project.state['sessions'] = self.session.project.state.get('sessions', []) + [self.session.id]
                
    def run_stim(self) -> None:

        self.update_state()
        
        if not np_services.ScriptCamstim.is_ready_to_start():
            raise RuntimeError("ScriptCamstim is not ready to start.")
        
        np_services.ScriptCamstim.script = self.camstim_script_path
        np_services.ScriptCamstim.params = self.camstim_params
        
        np_logging.web(f'vbn_{self.workflow.name.lower()}').info(f"Started camstim script")
        np_services.ScriptCamstim.start()
        
        with contextlib.suppress(Exception):
            while not np_services.ScriptCamstim.is_ready_to_start():
                time.sleep(2.5)
            
        if isinstance(np_services.ScriptCamstim, np_services.Finalizable):
            np_services.ScriptCamstim.finalize()

        with contextlib.suppress(Exception):
            np_logging.web(f'barcode_{self.workflow.name.lower()}').info(f"Finished session {self.mouse.mtrain.stage['name']}")
    
    
    def copy_data_files(self) -> None: 
        super().copy_data_files()
        
        # When all processing completes, camstim Agent class passes data and uuid to
        # /camstim/lims BehaviorSession class, and write_behavior_data() writes a
        # final .pkl with default name YYYYMMDDSSSS_mouseID_foragingID.pkl
        # - if we have a foraging ID, we can search for that
        if None == (stim_pkl := next(self.session.npexp_path.glob(f'{self.session.date:%y%m%d}*_{self.session.mouse}_*.pkl'), None)):
            logger.warning('Did not find stim file on npexp matching the format `YYYYMMDDSSSS_mouseID_foragingID.pkl`')
            return
        assert stim_pkl
        if not self.session.platform_json.foraging_id:
            self.session.platform_json.foraging_id = stim_pkl.stem.split('_')[-1]
        new_stem = f'{self.session.folder}.stim'
        logger.debug(f'Renaming stim file copied to npexp: {stim_pkl} -> {new_stem}')
        stim_pkl = stim_pkl.rename(stim_pkl.with_stem(new_stem))
        
        # remove other stim pkl, which is nearly identical, if it was also copied
        for pkl in self.session.npexp_path.glob('*.pkl'):
            if (
                self.session.folder not in pkl.stem
                and 
                abs(pkl.stat().st_size - stim_pkl.stat().st_size) < 1e6
            ):
                logger.debug(f'Deleting extra stim pkl copied to npexp: {pkl.stem}')
                pkl.unlink()
        
    def run_stim_desktop_theme_script(self, selection: str) -> None:     
        np_services.ScriptCamstim.script = '//allen/programs/mindscope/workgroups/dynamicrouting/ben/change_desktop.py'
        np_services.ScriptCamstim.params = {'selection': selection}
        np_services.ScriptCamstim.start()
        while not np_services.ScriptCamstim.is_ready_to_start():
            time.sleep(0.1)

    set_grey_desktop_on_stim = functools.partialmethod(run_stim_desktop_theme_script, 'grey')
    set_dark_desktop_on_stim = functools.partialmethod(run_stim_desktop_theme_script, 'dark')
    reset_desktop_on_stim = functools.partialmethod(run_stim_desktop_theme_script, 'reset')

            

class Hab(VBNMixin, np_workflows.PipelineHab):
    def __init__(self, *args, **kwargs):
        self.services = (
            np_services.MouseDirector,
            np_services.Sync,
            np_services.VideoMVR,
            self.imager,
            np_services.NewScaleCoordinateRecorder,
            np_services.SessionCamstim,
        )
        super().__init__(*args, **kwargs)


class Ephys(VBNMixin, np_workflows.PipelineEphys):
    def __init__(self, *args, **kwargs):
        self.services = (
            np_services.MouseDirector,
            np_services.Sync,
            np_services.VideoMVR,
            self.imager,
            np_services.NewScaleCoordinateRecorder,
            np_services.SessionCamstim,
            np_services.OpenEphys,
        )
        super().__init__(*args, **kwargs)


# --------------------------------------------------------------------------------------


def new_experiment(
    mouse: int | str | np_session.Mouse,
    user: str | np_session.User,
    workflow: Workflow = Workflow.PRETEST,
) -> Ephys | Hab:
    """Create a new experiment for the given mouse and user."""
    match workflow:
        case Workflow.PRETEST | Workflow.EPHYS:
            experiment = Ephys(mouse, user)
        case Workflow.HAB:
            experiment = Hab(mouse, user)
        case _:
            raise ValueError(f"Invalid workflow type: {workflow}")
    experiment.workflow = workflow
    
    with contextlib.suppress(Exception):
        np_logging.web(f'barcode_{experiment.workflow.name.lower()}').info(f"{experiment} created")
            
    return experiment



---
## Select workflow to run

In [ ]:
selected_workflow = Workflow.EPHYS
# selected_workflow = Workflow.HAB

***
## Generate new session
Check mouse ID and session are correct: this cell will lock them in!

In [ ]:
experiment: np_workflows.PipelineExperiment = new_experiment(mouse, user, selected_workflow)
session: np_session.PipelineSession = experiment.session
platform_json: np_session.PlatformJson = experiment.session.platform_json

platform_json.workflow_start_time = npxc.now()
hab: bool = isinstance(experiment, Hab)

In [ ]:
experiment.session.npexp_path.exists()

***
## Checks before starting

### **Ephys day 2?** 
Don't forget to adjust probe targeting!

In [ ]:
np_workflows.check_hardware_widget()

In [ ]:
if not hab:
    np_workflows.check_openephys_widget()

***
## Setup, test, reset all components
*This cell must not be skipped!*

In [ ]:
with contextlib.suppress(ZroError):
    experiment.initialize_and_test_services()

---
## MouseDirector: extend lick spout and set position for mouse
- so it doesn't fly out to an unknown position when the mouse is on the stage

***
## Dip probes

In [ ]:
if not hab:
    print(np_workflows.dye_info_widget.__doc__)
    np_workflows.dye_info_widget(session)

## Photodoc of probes in dye

In [ ]:
if not hab:
    print(str(session) + '_surface-image1-left.png')

## Probe depths in dye

In [ ]:
if not hab:
    np_workflows.probe_depth_widget(session)

***
***
# **With mouse on stage**
## Before lowering cartridge

In [ ]:
experiment.log('Mouse on stage')
platform_json.HeadFrameEntryTime = npxc.now()
np_workflows.wheel_height_widget(session)
np_workflows.check_mouse_widget()

***
## When cartridge is lowered

### Set zoom to 4.0 for photodocs of brain
- focus on the brain surface

## Photodoc of brain (tap probes if hab Day1 or Day2)

In [ ]:
platform_json.CartridgeLowerTime = npxc.now()
print(str(session) + '_surface-image2-left.png')

***
## ISI map

In [ ]:
np_workflows.isi_widget(mouse.lims)

***
## Probe insertion

In [ ]:
if not hab:
    platform_json.ProbeInsertionStartTime = npxc.now()

---
## Extra advance & retract each probe
- use NewScale GUI to advance an extra 100 $\mu m$ at 200 $\mu m/s$, then reverse 100 $\mu m$ at the same rate

***
## Photodoc before advancing probes

In [ ]:
if not hab:
    np_workflows.photodoc_widget('pre_insertion_surface_image')

***
## Settle timer & insertion notes & turn on laser

- run both cells now: settle timer will start

- fill out probe notes while waiting

- press Save once

- notes are saved when the timer finishes (button will turn green to confirm)

### *also turn on laser while waiting...*

In [ ]:
if not hab:
    np_workflows.insertion_notes_widget(session)

In [ ]:
if not hab:
    experiment.set_dark_desktop_on_stim()
    experiment.log('settle timer started')
    np_workflows.print_countdown_timer(minutes=.1 if experiment.workflow.value == 'pretest' else 10)
    experiment.log('settle timer finished')

***
## Photodoc after probes settled, before experiment

In [ ]:
if not hab:
    print(str(session) + '_surface-image4-left.png')

In [ ]:
np_workflows.pre_stim_check_widget()

***
## Start devices recording

In [ ]:
last_exception = Exception()
attempts = 3
while attempts:
    np_logging.getLogger().info('Waiting for recorders to finish processing') 
    while not all(r.is_ready_to_start() for r in experiment.recorders):
        time.sleep(1)
    np_logging.getLogger().info('Recorders ready')     
    try:
        experiment.start_recording()
    except AssertionError as exc:
        np_logging.getLogger().info('`experiment.start_recording` failed: trying again')
        attempts -= 1
        last_exception = exc              # exc only exists within the try block
    
    else:
        break
else:
    np_logging.getLogger().error(f'`experiment.start_recording` failed after multiple attempts', exc_info=last_exception)
    raise last_exception

***
## Start stimulus
mapping | main | opto 


Script and params determined by MTrain stage

In [ ]:
np_services.MouseDirector.get_proxy().extend_lick_spout()

In [ ]:
with contextlib.suppress(ZroError):
    experiment.run_stim()

***
## Stop recording

In [ ]:
with contextlib.suppress(ZroError):
    experiment.stop_recording_after_stim_finished()

In [ ]:
np_services.MouseDirector.get_proxy().retract_lick_spout()
experiment.reset_desktop_on_stim()

***
## Before removing probes

In [ ]:
if not hab:
    print(str(session) + '_surface-image5-left.png')

***
## After fully retracting probes

In [ ]:
if not hab:    
    print(str(session) + '_surface-image6-left.png')

***
## After raising cartridge

In [ ]:
platform_json.HeadFrameExitTime = npxc.now()

np_workflows.finishing_checks_widget()

## Finalize

In [ ]:
platform_json.workflow_complete_time = npxc.now()

experiment.finalize_services(*experiment.recorders, *experiment.stims)
experiment.validate_services(*experiment.recorders, *experiment.stims)

## Copy data

In [ ]:
experiment.copy_files()

# Add to post-experiment pipeline

**hab**
- add session to QC queue

**ephys**
- add session to np-exp upload queue, specifying this rig's Acq as `hostname`
    - ensures checksum-validated copy of ephys data on np-exp
    - then adds session to spike-sorting queue
    - then adds session to QC queue

    
    #### run *"process sorting queue .exe"* on Acq desktop

In [ ]:
if hab:
    np_jobs.PipelineQCQueue().add_or_update(session, priority=99)
else:
    np_jobs.PipelineNpexpUploadQueue().add_or_update(session, hostname=np_config.Rig().Acq, priority=99)